# Modeling notebook

In [1]:
import pandas as pd
import numpy as np

testing_models = False

# TODO:
# data managment make seperate notebooks for some of these
# Add forecasting models that work
# Determine when a forecasting model works given a time series features.

In [2]:
%run setup.ipynb

# tune models on a subset of categories

In [3]:

# subset for testing

if testing_models:
    train = train[train.store_item.isin(['1-1', '1-2', '1-3'])]
    test = test[test.store_item.isin(['1-1', '1-2', '1-3'])]
else: 
    from warnings import filterwarnings
    filterwarnings('ignore')
    

# vector autoreg

In [4]:
from statsmodels.tsa.ar_model import AutoReg

In [5]:
# generalize this a bit later, 
preds_autoreg = []
store_item = []
for series in train.store_item.unique(): 
    temp = train[train.store_item == series]
    
    model = AutoReg(temp.sales, lags=10, old_names=False, seasonal=True, period=365)
    
    try:
        model_fit = model.fit()
        yhat = model_fit.predict(start = test.index[0], end = test.index[-1])
        preds_autoreg.append(yhat)
        item = np.repeat(series, len(yhat))
        store_item.append(item)
    except np.linalg.LinAlgError:
        print(f'series {series}')

/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/

In [6]:
dates = []
for item in preds_autoreg:
    for sub in item.index:
        dates.append(sub)

In [7]:
predictions = pd.DataFrame(
    {
        'store_item': [x for sub in store_item for x in sub],
        'sales': test.sales,
        'autoreg': [x for sub in preds_autoreg for x in sub]     
    }, index = dates
)

# exp smoothing

In [8]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [9]:
def exp_smooth_predictor(seas="multiplicative"):
    exp_smooth_preds = []
    for item in train.store_item.unique():
        temp = train[train.store_item == item]
        # temp.index.freq = "d"
        preds = ExponentialSmoothing(temp.sales,     
            seasonal_periods=365,
            trend="add",
            seasonal=seas,
            use_boxcox=True,
            initialization_method="estimated")\
        .fit()\
        .predict(start = test.index[0], end = test.index[-1])
        exp_smooth_preds.append(preds)

            
    return [x for sub in exp_smooth_preds for x in sub]  

In [10]:
predictions["exp_smooth"] = exp_smooth_predictor(seas="add")
predictions["exp_smooth_multi"] = exp_smooth_predictor()

/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/

6-4


/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/

ValueError: Length of values (272406) does not match length of index (272953)

# ardl autoregressive distributed lag

In [ ]:
from statsmodels.tsa.api import ARDL

In [ ]:
ardl_preds = []

for item in train.store_item.unique():
    temp = train[train.store_item == item]
    ar = ARDL(temp.sales, 365, period=365, trend="t")\
        .fit()\
        .predict(start = test.index[0], end = test.index[-1])
    ardl_preds.append(ar)
    
predictions["ardl"] = [x for sub in ardl_preds for x in sub]  

/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:152: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and NumericIndex with a unit increment support extending. The index is set will contain the position relative to the data length.
  warnings.warn(
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/Users/josephdespres/my_env/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:152: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and NumericIndex with a uni

# prophet model

In [ ]:
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [ ]:
# https://www.youtube.com/watch?v=pOYAXv15r3A
model = Prophet()


In [ ]:
forecast_h = test.index[-1] - test.index[0]

In [ ]:
prophet_preds = []
for item in train.store_item.unique():
    
    tester = train[train.store_item == item]
    tester.reset_index(inplace=True)

    tester["unique_id"] = item
    tester.drop("store_item", axis=1, inplace=True)
    tester.rename(columns={"sales":"y", "date":"ds"}, inplace=True)
    
    model = Prophet(daily_seasonality=True)
    model.fit(tester)
    pred_frame = model.make_future_dataframe(periods=forecast_h.days + 1, include_history=False)
    preds = model.predict(pred_frame)
    prophet_preds.append(preds.yhat)


/var/folders/0n/gp39qxl100149ccbr1n65kzh0000gn/T/ipykernel_4442/1183488422.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tester["unique_id"] = item
/Users/josephdespres/my_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/josephdespres/my_env/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

Initial log joint probability = -23.8441
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      94       2304.99    0.00059656       212.127    4.31e-06       0.001      145  LS failed, Hessian reset 
      99       2305.17   0.000825136       89.4793      0.9746      0.9746      150   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     193        2305.9    0.00037808       161.031   2.206e-06       0.001      316  LS failed, Hessian reset 
     199       2306.19    0.00069264       73.7016      0.3052           1      323   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     243       2306.29   4.25779e-05       86.2633   5.088e-07       0.001      415  LS failed, Hessian reset 
     271       2306.29   1.51602e-07       79.5501           1           1      452   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is

/var/folders/0n/gp39qxl100149ccbr1n65kzh0000gn/T/ipykernel_4442/1183488422.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tester["unique_id"] = item
/Users/josephdespres/my_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/josephdespres/my_env/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

Initial log joint probability = -34.5062
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2717.63   0.000201329       78.4211      0.7713      0.7713      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2721.23   0.000657679       64.9873      0.4638           1      262   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     266       2721.27   2.03817e-05       61.8377   4.043e-07       0.001      393  LS failed, Hessian reset 
     299       2721.27   6.35448e-06       61.8897       1.593      0.4364      432   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     365       2721.28   1.48875e-07       76.8861      0.3067           1      514   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/var/folders/0n/gp39qxl100149ccbr1n65kzh0000gn/T/ipykernel_4442/1183488422.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tester["unique_id"] = item
/Users/josephdespres/my_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/josephdespres/my_env/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

Initial log joint probability = -20.7053
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      98       2482.26   0.000209279       112.769   1.462e-06       0.001      192  LS failed, Hessian reset 
      99       2482.31    0.00124962       83.2433          10           1      194   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     114        2482.4   9.02554e-05       51.7204   1.639e-06       0.001      247  LS failed, Hessian reset 
     136       2482.41   2.62744e-05        85.761   3.638e-07       0.001      312  LS failed, Hessian reset 
     159       2482.41   1.24874e-07       61.4959      0.8433      0.8433      349   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [ ]:
predictions["prophet"] = [x for sub in prophet_preds for x in sub] 

# xgboost

In [ ]:
from xgboost import XGBRegressor
# pip install xgboost==0.80
# the latest version kept crashing on me

In [ ]:
def create_features(df):
    df['date'] = df.index
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    
    return X

In [ ]:
reg = XGBRegressor(n_estimators=1000)

xgb_preds = []
for item in train.store_item.unique():
    tmp = train[train.store_item == item]
    test_df = test[test == item] # this is not cheating cuz we will know days and such
    
    feat_frame = create_features(tmp)
    tester = create_features(test_df)
    
    reg.fit(feat_frame, tmp.sales)
    all_preds = reg.predict(tester)
    xgb_preds.append(all_preds[-forecast_h.days-1:])

/var/folders/0n/gp39qxl100149ccbr1n65kzh0000gn/T/ipykernel_4442/2217153910.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df.index
/var/folders/0n/gp39qxl100149ccbr1n65kzh0000gn/T/ipykernel_4442/2217153910.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayofweek'] = df['date'].dt.dayofweek
/var/folders/0n/gp39qxl100149ccbr1n65kzh0000gn/T/ipykernel_4442/2217153910.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [ ]:
predictions["xgb_preds"] = [x for sub in xgb_preds for x in sub] 

In [ ]:
store_sales = pd.concat([train, predictions])

In [ ]:
from datetime import date

today = date.today()

In [ ]:
predictions.to_csv(f"../data/predictions/predictions-{today.month}-{today.day}.csv")

# plot results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

def fcastplotter(x):
        
    for store in store_sales.store_item.unique():
        temp = store_sales[store_sales.store_item == store]
        plt.figure()
        plt.plot(temp.sales, "-b", label = "sales")
        plt.plot(temp[x],  "-r", label = f"Forecast {x} ")
        plt.legend(loc="upper left")
        plt.ylim([0, None])
        plt.title(f"store {store}")
        plt.show()

In [ ]:
# plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

# for forecast in store_sales.columns[2:]:
#     fcastplotter(forecast)


NameError: name 'plt' is not defined

In [ ]:
from beepy import beep
beep()